In [1]:
import time, json
import requests
from bs4 import BeautifulSoup

In [2]:
def read_api_key(filepath):
    with open(filepath, 'r') as f:
        return f.read().replace('\n','')
    
api_key = read_api_key('api_key.txt')
print(api_key)

yfmkhh2aWg8dnQyvnoRXg


In [3]:
# Test function for querying goodreads API
def goodreads_query(api_key, url, query):
    """
    api_key (string): API key
    url (string): Base query URL
    query (dictionary): Query parameters
    """
    
    request_params = query
    request_params['key'] = api_key
    response = requests.get(url, params=request_params)
    
    return response
    
temp_response = goodreads_query(api_key, 'https://www.goodreads.com/book/show', {'id':10572, 'format':'xml', 'text_only':'true'})

In [13]:
# Get all the reviews for a given book ID as a list of dictionaries
# Review data we are collecting - rating, date, and review text
def get_reviews(api_key, book_id):
    request_params = {'id':book_id, 'key':api_key, 'format':'xml', 'text_only':'true'}
    response = requests.get('https://www.goodreads.com/book/show', params=request_params)
    
    soup = BeautifulSoup(response.text, 'lxml')
    iframe = soup.find('reviews_widget').find('iframe')
    
    all_reviews = []
    
    reviews_url = iframe['src']
    response_reviews = requests.get(reviews_url)
    
    while(1):
        review_soup = BeautifulSoup(response_reviews.text, 'lxml')
        reviews = review_soup.find_all('div', {'class':'gr_review_container'})

        for review in reviews:
            current_review = {}

            # Extract the rating
            rating = review.find('span', {'class':'gr_rating'})
            if (rating):
                rating = rating.find_all(text=True)
                rating_num = 0
                for i in range(len(rating[0])):
                    if (ord(rating[0][i]) == 9733):
                        rating_num += 1
            else:
                rating_num = -1

            # Extract the date
            date = review.find('span', {'class':'gr_review_date'}).find_all(text=True)
            date = [x.strip() for x in date][0]

            # Extract the review text
            s = [x.strip() for x in review.find('div', {'class':'gr_review_text'}).find_all(text=True)]
            s = [x for x in s if x]

            # Ignore the last element, the '...more'
            review_text = ' '.join(s[:-1])

            # Remove the last word, since it will be partial
            review_text = ' '.join(review_text.split(' ')[:-1])

            current_review['rating'] = rating_num
            current_review['date'] = date
            current_review['text'] = review_text

            all_reviews.append(current_review)

            print(current_review)
        
        # See if there is another page of reviews
        if (review_soup.find_all('a', {'class':'next_page'}) != []):
            # There is another page
            reviews_url = 'https://goodreads.com' + review_soup.find('a', {'class':'next_page'})['href']
            response_reviews = requests.get(reviews_url)
        else:
            # No more pages of reviews
            break
        
    return all_reviews

    
book_reviews = get_reviews(api_key, 68428)
print(len(book_reviews))

{'rating': 5, 'date': 'January 28, 2015', 'text': "BOOKTALK HERE: https://www.youtube.com/watch?v=C-XEW... Just as I expected, this book was FANTASTIC. The story, the characters, the world, the magic system--I loved it all! It was a wonderful first book in a series--satisfying on its own, yet there's still plenty to explore in future books"}
{'rating': 4, 'date': 'December 30, 2015', 'text': 'Sometimes, I revisit a book I had loved ages ago, only to tell myself "What the actual fuck was I thinking?! This book is crap!!!!! Oh young Khanh, you were so dumb :(" Thankfully, this was absolutely NOT the case for this book. I appreciated it much more the second time around. It could be that'}
{'rating': 3, 'date': 'December 12, 2016', 'text': "I can't remember being this violently conflicted about a book in quite some time. There are some areas where it's just so well done, with the author absolutely nailing it, and then others where I found myself grinding my teeth in frustration. I'm going 

{'rating': 4, 'date': 'December 16, 2017', 'text': "Where do I even begin ? THIS BOOK WAS PERFECT. WHERE ARE THE FLAWS ? I CAN'T FIND ONE. When I started this book, I had no idea what it was about. I just knew people loved it, and that's all. And I wanted to love it as well. SO badly. AND I DID. I loved the magic system. You know in some books you"}
{'rating': 5, 'date': 'December 17, 2013', 'text': 'WOW. That. Was. Really.'}
{'rating': 4, 'date': 'August 26, 2015', 'text': 'Reread of Series starting August 12 - Just as good the 2nd time through plus I caught some hints this time so even better. Shadows of Self should be released October 6th. So what better excuse do I need to reread every book in the Mistborn world? NONE!!! Original Review May 2014: “There’s always'}
{'rating': 5, 'date': 'February 20, 2017', 'text': "Well, for starters, this was the best prologue of any book I've ever read. I have been neglecting this series (and author) for far too long, and after I read that prolog

{'rating': 4, 'date': 'April 21, 2018', 'text': "Actual rating : 4.5 stars My reaction after ending the book - Finally I've read a Sanderson book and I totally get the hype! It was very well written, provided so much action, offered a world full of depth and provided so many thought provoking moments of love, loss, life, and hope. Highlights :"}
{'rating': 5, 'date': 'June 05, 2013', 'text': 'Epic Buddy Read with my Epic Fantasy loving buddy readers Tom, Amy (BW), Pat, Rosanna, Sarah (my lovely), Estelle, and Roxanne...Whew!!! I\'ve been dying to read this series!!!!!!!!! ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~ "Our belief is often strongest when it should be weakest. That is the nature'}
{'rating': 5, 'date': 'March 24, 2017', 'text': "My eyes burn! I've spend the last 24h hours reading the second half of this book. It's a masterpiece and I couldn't be more in love with a book series. Recommend forever! Check my review below on my blog with all my thoughts and comment what you think abou

{'rating': 5, 'date': 'March 04, 2018', 'text': "Buddy read with Buddies Books and Baubles . This book is a must-read. I'm left with a Dune feeling after reading it. I feel using gifs to review it would be simply to frivolous. The story is quite simple: take an immortal emperor, add slaves, an insane genius and good people who see beyond"}
{'rating': 5, 'date': 'August 11, 2015', 'text': "Okay, okay. First off, I'm a bit of a Sanderson newbie. My first foray into Brandon's remarkable mind came at the end of last year, where despite having started The Way of Kings 3 or 4 times, Steelheart became the first Sanderson story I would ever finish. Now I'm a Sanderson Fangirl. I have"}
{'rating': 5, 'date': 'December 09, 2017', 'text': '"I worry about what they will say of me. Historians can make what they want of the past. In a thousand years\' time, will I be remembered as the man who protected mankind from a powerful evil? Or, will I be remembered as a tyrant who arrogantly tried to make hi